In [1]:
#Tested in Python version 3.8

###import libraries
from pathlib import Path
import os
import os.path
import datetime
import time
import re
import shutil
import sys
import glob
import argparse
from urllib.parse import urljoin
from io import StringIO
import pandas as pd
import numpy as np
from osgeo import gdal
import pyproj
import rasterio
from rasterio.crs import CRS
from rasterio.warp import calculate_default_transform, reproject, Resampling

#convert from julian calendar to standard  calendar and vv
def jdtodatestd (jdate):
    fmt = '%Y%j'
    datestd = datetime.datetime.strptime(jdate, fmt).date()
    return(datestd)

#convert from standard calendar to julian calendar
def datestdtojd (stddate):
    fmt='%Y-%m-%d'
    sdtdate = datetime.datetime.strptime(stddate, fmt)
    sdtdate = sdtdate.timetuple()
    jdate = sdtdate.tm_yday
    return(jdate)

#sets-up URL for download    
def downloadh5_dates(start_date, end_date, loc_dir, h5_folder, token, h5_list):
    delta = datetime.timedelta(days=1)
    
    julian_day_start = datestdtojd(str(start_date.year) + '-' + str(start_date.month) + '-' + str(start_date.day)) 
    
    #make this more generic  "https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MOD13Q1"
    year_URL = urljoin("https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MOD21A2/", str(start_date.year))
    try:
        import csv
        days = [ f for f in csv.DictReader(StringIO(geturl('%s.csv' % year_URL, token)), skipinitialspace=True) ] #gets csv from website
    except ImportError:
        import json
        days = json.loads(geturl(year_URL + '.json', token))
    
    validDays_year = start_date.year
    validDays = [] #get list of valid days
    
    for day in days:
        validDays.append(day['name'])
     
    #loops between start date and end date
    while start_date <= end_date:
                
        #assigns folder prefix
        julian_day = datestdtojd(str(start_date.year) + '-' + str(start_date.month) + '-' + str(start_date.day))
        if julian_day < 10:
            day_string = "00"+ str(julian_day)
        elif julian_day < 100:
            day_string = "0"+ str(julian_day)
        else:
            day_string = str(julian_day)

        #creates source URL for h5 files
        source_URL = urljoin("https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MOD21A2/", str(start_date.year) + "/" + day_string +"/") 
#         print(source_URL)


        if day_string in validDays:
            print(str(start_date) + " with available image.")
            
            #creates location folder path where h5 will be downloaded
            dest = loc_dir / h5_folder  / (str(start_date.year) + day_string)

            if not os.path.exists(dest): #checks if h5_folder exists, if not it will be created
                os.makedirs(dest)
            
            downloadh5(source_URL, dest, token, h5_list) #calls download function of h5 files from NASA server
        else:
            print(str(start_date) + " with no available image.")
            
        start_date += delta #moves to next day  
        
        #for cases of transition from one year to the next, we need to update valid dates
        if (start_date.year != validDays_year):
            year_URL = urljoin("https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MOD21A2/", str(start_date.year))
            try:
                import csv
                days = [ f for f in csv.DictReader(StringIO(geturl('%s.csv' % year_URL, token)), skipinitialspace=True) ] #gets csv
            except ImportError:
                import json
                days = json.loads(geturl(year_URL + '.json', token))

            validDays_year = start_date.year
            validDays = []
            for day in days:
                validDays.append(day['name'])
                       
    
#access online source and formats destination files
def downloadh5(src, dest, tok, h5):
    '''synchronize src url with dest directory'''

    try:
        import csv
        files = [ f for f in csv.DictReader(StringIO(geturl('%s.csv' % src, tok)), skipinitialspace=True) ] #gets csv
    except ImportError:
        import json
        files = json.loads(geturl(src + '.json', tok)) #gets json

    # use os.path since python 2/3 both support it while pathlib is 3.4+
    for f in files:
        # currently we use filesize of 0 to indicate directory
        start = time.perf_counter()
        filesize = int(f['size'])
        path = os.path.join(dest, f['name'])
        url = src + '/' + f['name']
        if filesize == 0:
            try:
                print('creating dir:', path)
                os.mkdir(path)
                sync(src + '/' + f['name'], path, tok)
            except IOError as e:
                print("mkdir `%s': %s" % (e.filename, e.strerror), file=sys.stderr)
                sys.exit(-1)
        else:
            try:
                if not os.path.exists(path):
                    if f['name'][17:23] in h5:
                        print('Downloading: ' , url) #path
                        with open(path, 'w+b') as fh:
                            geturl(url, tok, fh)
                        end = time.perf_counter() - start
                        print("Finished in %.4f seconds." %(time.perf_counter() - start))
                else:
                    print('skipping: ', f['name'])
            except IOError as e:
                print("open `%s': %s" % (e.filename, e.strerror), file=sys.stderr)
                sys.exit(-1)
    return 0

USERAGENT = 'tis/download.py_1.0--' + sys.version.replace('\n','').replace('\r','')

#sets up urlib options
def geturl(url, token=None, out=None):
    headers = { 'user-agent' : USERAGENT }
    if not token is None:
        headers['Authorization'] = 'Bearer ' + token
    try:
        import ssl
        ssl._create_default_https_context = ssl._create_unverified_context
        CTX = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
        if sys.version_info.major == 2:
            import urllib2            
            
            try:
                fh = urllib2.urlopen(urllib2.Request(url, headers=headers), context=CTX)
                if out is None:
                    return fh.read()
                else:
                    shutil.copyfileobj(fh, out)
            except urllib2.HTTPError as e:
                print('HTTP GET error code: %d' % e.code(), file=sys.stderr)
                print('HTTP GET error message: %s' % e.message, file=sys.stderr)
            except urllib2.URLError as e:
                print('Failed to make request: %s' % e.reason, file=sys.stderr)
            return None

        else:
            from urllib.request import urlopen, Request, URLError, HTTPError
            try:
                fh = urlopen(Request(url, headers=headers), context=CTX)
                if out is None:
                    return fh.read().decode('utf-8')
                else:
                    shutil.copyfileobj(fh, out)
            except HTTPError as e:
                print('HTTP GET error code: %d' % e.code, file=sys.stderr)
                print('HTTP GET error message: %s' % e.message, file=sys.stderr)
            except URLError as e:
                print('Failed to make request: %s' % e.reason, file=sys.stderr)
            return None

    except AttributeError:
        # OS X Python 2 and 3 don't support tlsv1.1+ therefore... curl
        import subprocess
        try:
            args = ['curl', '--fail', '-L', '--get', url, '--ssl-no-revoke']
            for (k,v) in headers.items():
                args.extend(['-H', ': '.join([k, v])])
            if out is None:
                # python3's subprocess.check_output returns stdout as a byte string
                result = subprocess.check_output(args)
                return result.decode('utf-8') if isinstance(result, bytes) else result
            else:
                subprocess.call(args, stdout=out)
        except subprocess.CalledProcessError as e:
            print('curl GET error message: %' + (e.message if hasattr(e, 'message') else e.output), file=sys.stderr)
        return None

def hdfToGeotiff(src_h5, destination):

    #Sets geotiff prefix and suffix
    rasterFilePre = str(src_h5.name).replace('.hdf','')
    fileExtension = ".tif"

    #Open HDF file
    hdflayer = gdal.Open(str(src_h5), gdal.GA_ReadOnly)  
    
    if not hdflayer is None:

        #Loops/open bands of h5 file
        for i in hdflayer.GetSubDatasets():
            if i[1] in ['[1200x1200] LST_Day_1KM MODIS_Grid_8Day_1km_LST21 (16-bit unsigned integer)','[1200x1200] QC_Day MODIS_Grid_8Day_1km_LST21 (8-bit unsigned integer)']:    
                sublayer = i[1].replace('"','').replace(' ','_')
                rlayer = gdal.Open(i[0], gdal.GA_ReadOnly)
                geotransform = rlayer.GetGeoTransform()

                if geotransform:
                    x_size = rlayer.RasterXSize
                    y_size = rlayer.RasterYSize
                    pixelsize_x = geotransform[1]
                    pixelsize_y = geotransform[5]
                    WestBoundCoord = geotransform[0]
                    NorthBoundCoord = geotransform[3]
                    EastBoundCoord = WestBoundCoord + (x_size * pixelsize_x)
                    SouthBoundCoord = NorthBoundCoord +  (y_size * pixelsize_y)

              #Output file name editing
                outputNameFinal =  rasterFilePre + "_" + sublayer + fileExtension

                try:
                    outputRaster = destination + "/" + outputNameFinal
                except:
                    outputRaster = str(destination / outputNameFinal)

                if not os.path.exists(outputRaster):
                    #assign projections
                    try:
                        EPSG = "ESRI:53008" #Sinusoidal
                    except:
                        EPSG = "-a_srs ESRI:53008"
                        
                    #returns error , if any
                    gdal.UseExceptions()
                    
                    #Process output geotiffs
                    translateOptionText = EPSG +" -a_ullr " + str(WestBoundCoord) + " " + str(NorthBoundCoord) + " " + str(EastBoundCoord) + " " + str(SouthBoundCoord)
                    translateoptions = gdal.TranslateOptions(gdal.ParseCommandLine(translateOptionText))
                    gdal.Translate(outputRaster,rlayer, options=translateoptions)
                    
                else:
                    print('skipping: ', outputNameFinal)


def mask(evi, vi_q):
    
    output = str(evi.name).replace("_[1200x1200]_LST_Day_1KM_MODIS_Grid_8Day_1km_LST21_(16-bit_unsigned_integer)","").replace(".tif","_Masked.tif")
    if not os.path.exists(evi.parent/output):
        #returns 1 if mask value is satisfied otherwise returns 0
        def mask_fnc(value):
            #adjust mask conditional statement here
            if value % 4 == 0:             
                return 1.0
            else:
                return np.nan
        
        #reads VI Quality geotiff as rasterio object
        a = np.array(rasterio.open(vi_q).read(1))
    
        #call mask_fnc for 0-1 value
        binary_mask = np.vectorize(mask_fnc)
        mask = binary_mask(a).astype(rasterio.float32)
        
        #reads EVI geotiff as rasterio object
        evi_image = np.array(rasterio.open(evi).read(1))
        
        masked = evi_image * mask
    
        #sets raster metadata for file writing
        with rasterio.open(evi) as raster:
            meta = raster.meta
            filename = evi.parent/output
        meta.update(dtype=rasterio.float32)            
    
        #writes the mask file
        with rasterio.open(filename, "w", **meta) as filename:
            filename.write(masked.astype(rasterio.float32),1)
    else:
        print('skipping: ', evi.name)
        
def reprojectRaster(inputRaster,outputRaster):

    if not os.path.exists(outputRaster):
        #open source raster
        srcRst = rasterio.open(inputRaster)
        # dstCrs = {'EPSG:4326'}
        try:
            dstCrs = rasterio.CRS({'init': 'epsg:4326'})
        except:
            # dstCrs = {'init': 'epsg:4326'}
            dstCrs = CRS.from_epsg(4326)
    
        transform, width, height = calculate_default_transform(srcRst.crs, dstCrs, srcRst.width, srcRst.height, *srcRst.bounds)
    
        kwargs = srcRst.meta.copy()
        kwargs.update({
                'crs': dstCrs,
                'transform': transform,
                'width': width,
                'height': height
            })
        #open destination raster
        dstRst = rasterio.open(outputRaster, 'w', **kwargs)
        
        #reproject and save raster band data
        for i in range(1, srcRst.count + 1):
            reproject(
                source=rasterio.band(srcRst, i),
                destination=rasterio.band(dstRst, i),
                # src_transform=srcRst.transform,
                src_crs=srcRst.crs,
                # dst_transform=transform,
                dst_crs=dstCrs,
                resampling=Resampling.nearest)
        
        #close rasters
        srcRst.close()
        dstRst.close()
    else:
        print('skipping: ', outputRaster)
        
    
###set-up token and local directories
token = "" #insert your token here from EarthData


loc_dir = Path("C:/Users/Homer/OneDrive/Documents/01 Projects/28 HKUST Urban Heat/GitHub Test") 
hdf_folder = "01 HDF" #folder within working directory where hdfs will be downloaded
geotiff_folder = "02 Geotiff" #folder within working directory where geotiff will be saved


###edit date details
first_date = datetime.date(2024, 6, 1) #user input - format: (year, month, day) no preceding zero e.g. use '1' instead of '01'
last_date = datetime.date(2024, 6, 30) #user input - format: (year, month, day) no preceding zero e.g. use '1' instead of '01'


#creates Path for download folder
hdf_dir = loc_dir/ hdf_folder
geotiff_dir = loc_dir/ geotiff_folder

# tiles_list = list(tileLister(loc_dir/'MODISTiles.csv' shp_dir ))
tiles_csv = "TileList_Manila.csv"
tiles = pd.read_csv(loc_dir/ tiles_csv)
tiles_list = tiles['TileID'].tolist()


print("Processing MOD21A2 LST from", first_date, "to", last_date)   
    
#Step 1 Downloading of hdf files from NASA server
if not os.path.exists(hdf_dir): #checks if folder exists if not it will be created
    os.makedirs(hdf_dir)   
downloadh5_dates(first_date,last_date,loc_dir,hdf_folder,token,tiles_list)

#Step 2 Extraction and conversion of hdf files to geotiff format
if not os.path.exists(geotiff_dir): #checks if folder exists if not it will be created
    os.makedirs(geotiff_dir)
paths = hdf_dir.glob('**/*.hdf')
for path in paths:
    print("Extracting: ", path.name)
    hdfToGeotiff(path, geotiff_dir)

#Step 3 Masking using quality layer
paths = geotiff_dir.glob('**/*LST_Day_1KM_MODIS_Grid_8Day_1km_LST21_(16-bit_unsigned_integer).tif')
for path in paths:
    print("Masking: ", path.name)
    mask(path, path.parent / str(path.name).replace('LST_Day_1KM_MODIS_Grid_8Day_1km_LST21_(16-bit_unsigned_integer)','QC_Day_MODIS_Grid_8Day_1km_LST21_(8-bit_unsigned_integer)'))

#Step 4 Reprojects images to WGS84
paths = geotiff_dir.glob('**/*_Masked.tif')
for path in paths:
    print("Reprojecting to WGS84:", path.name)
    reprojectRaster(path, str(path).replace("_Masked.tif", "_Masked_WGS84.tif"))

Processing MOD21A2 LST from 2024-06-01 to 2024-06-30
2024-06-01 with available image.
Downloading:  https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MOD21A2/2024/153//MOD21A2.A2024153.h29v07.061.2024162203423.hdf
Finished in 2.6435 seconds.
2024-06-02 with no available image.
2024-06-03 with no available image.
2024-06-04 with no available image.
2024-06-05 with no available image.
2024-06-06 with no available image.
2024-06-07 with no available image.
2024-06-08 with no available image.
2024-06-09 with available image.
Downloading:  https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MOD21A2/2024/161//MOD21A2.A2024161.h29v07.061.2024175161830.hdf
Finished in 2.7174 seconds.
2024-06-10 with no available image.
2024-06-11 with no available image.
2024-06-12 with no available image.
2024-06-13 with no available image.
2024-06-14 with no available image.
2024-06-15 with no available image.
2024-06-16 with no available image.
2024-06-17 with available image.
Downloading:  ht